In [2]:
import json
import numpy as np
import pandas as pd
from datetime import datetime

In [3]:
outputdir = '/data/atomic_traffic_24h'

dataname = 'vo_sens'

dataset = pd.read_csv(dataname+'.csv')


rel = []
reldict = dict()
# dataset = pd.read_csv(dataurl+'PEMS04.csv')
for i in range(dataset.shape[0]):
    sid = dataset['from'][i]
    eid = dataset['to'][i]
    cost = dataset['cost'][i]
    if (sid, eid) not in reldict:
        reldict[(sid, eid)] = len(reldict)
        rel.append([len(reldict) - 1, 'geo', sid, eid, cost])
rel = pd.DataFrame(rel, columns=['rel_id', 'type', 'origin_id', 'destination_id', 'cost'])
rel.to_csv(dataname+'.rel', index=False)


dataset = np.load('traffic_9_10_11.npz')['arr_0']  # (16992, 307, 3)


start_time = datetime.strptime('2023-09-01T00:00:00Z', "%Y-%m-%dT%H:%M:%SZ")
end_time = datetime.strptime("2023-11-13T23:59:00Z","%Y-%m-%dT%H:%M:%SZ")


timeslot = []
work_hours = 0 # с 9:00 до 23:00 (15 часов)
while start_time < end_time:
    timeslot.append(start_time)
    start_time = datetime.fromtimestamp(datetime.timestamp(start_time) + 3600)

# dyna = []
dyna_id = 0
dyna_file = open(dataname+'.dyna', 'w')
dyna_file.write('dyna_id' + ',' + 'type' + ',' + 'time' + ',' + 'entity_id'
                + ',' + 'traffic_speed' + '\n')
for j in range(dataset.shape[1]):
    entity_id = j  # 这个数据集的id是0-306
    for i in range(len(timeslot)):
        time = timeslot[i]
        # dyna.append([dyna_id, 'state', time, entity_id, dataset[i][j][0], dataset[i][j][1], dataset[i][j][2]])
        dyna_file.write(str(dyna_id) + ',' + 'state' + ',' + str(time)
                        + ',' + str(entity_id) + ',' + str(dataset[i][j]) + '\n')
        dyna_id = dyna_id + 1
        if dyna_id % 10000 == 0:
            print(str(dyna_id) + '/' + str(dataset.shape[0]*dataset.shape[1]))
dyna_file.close()
# dyna = pd.DataFrame(dyna, columns=['dyna_id', 'type', 'time', 'entity_id', 'traffic_flow'])
# dyna.to_csv(dataname+'.dyna', index=False)


config = dict()
config['geo'] = dict()
config['geo']['including_types'] = ['Point']
config['geo']['Point'] = {}
config['rel'] = dict()
config['rel']['including_types'] = ['geo']
config['rel']['geo'] = {'cost': 'num'}
config['dyna'] = dict()
config['dyna']['including_types'] = ['state']
config['dyna']['state'] = {'entity_id': 'geo_id', 'traffic_speed': 'num'}
config['info'] = dict()
config['info']['data_col'] = ['traffic_speed']
config['info']['weight_col'] = 'cost'
config['info']['data_files'] = ['vo_sens']
config['info']['geo_file'] = 'vo_sens'
config['info']['rel_file'] = 'vo_sens'
config['info']['output_dim'] = 3
config['info']['time_intervals'] = 3600
config['info']['init_weight_inf_or_zero'] = 'zero'
config['info']['set_weight_link_or_dist'] = 'link'
config['info']['calculate_weight_adj'] = False
config['info']['weight_adj_epsilon'] = 0.1
json.dump(config, open('config.json', 'w', encoding='utf-8'), ensure_ascii=False)

FileNotFoundError: [Errno 2] No such file or directory: 'vo_sens.csv'